In [ ]:
import yasp

In [ ]:
import numpy as np
import array


import heppyy.util.fastjet_cppyy
import heppyy.util.heppyy_cppyy

from cppyy.gbl import fastjet as fj
from cppyy.gbl.std import vector
from cppyy.gbl import EnergyCorrelators

from heppyy.util.mputils import logbins
from heppyy.util.treewriter import RTreeWriter


In [ ]:
pythia_config = [
    'HardQCD:all = on',
    'Next:numberCount = 0',
    'Next:numberShowEvent = 0',
    'Next:numberShowInfo = 0',
    'Next:numberShowProcess = 0',
    'PhaseSpace:pTHatMin = 100.00000'
]

stable_charm = False
if stable_charm:
	for c in [411,413,421,423,431,433]:
		pythia_config.append(f'{c}:mayDecay=false')
		pythia_config.append(f'-{c}:mayDecay=false')

stable_beauty = False
if stable_beauty:
	for c in [511,513,521,523,531,533]:
		pythia_config.append(f'{c}:mayDecay=false')
		pythia_config.append(f'-{c}:mayDecay=false')


In [ ]:
import heppyy.util.pythia8_cppyy
from cppyy.gbl import Pythia8

In [ ]:
pythia = Pythia8.Pythia()
_ = [pythia.readString(s) for s in pythia_config]

In [ ]:
pythia.init()

In [ ]:
jet_R0 = 0.4
jet_etamax = 2.
jet_ptmin = 100.
jet_ptmax = 120.

hadron_etamax = jet_etamax + jet_R0 * 1.05

jet_def = fj.JetDefinition(fj.antikt_algorithm, jet_R0)
jet_selector = fj.SelectorPtMin(jet_ptmin) * fj.SelectorPtMax(jet_ptmax) * fj.SelectorAbsEtaMax(hadron_etamax - jet_R0 * 1.05)
jet_def_lund = fj.JetDefinition(fj.cambridge_algorithm, 1.0)
lund_gen = fj.contrib.LundGenerator(jet_def_lund)


In [ ]:
nevents = 1000

from tqdm.notebook import tqdm
all_jets = []
for i in tqdm(range(nevents)):
    if not pythia.next():
        continue
    # parts = vector[fj.PseudoJet]([fj.PseudoJet(p.px(), p.py(), p.pz(), p.e()) for p in pythia.event if p.isFinal() and p.isCharged()])
    parts = vector[fj.PseudoJet]([fj.PseudoJet(p.px(), p.py(), p.pz(), p.e()) for p in pythia.event if p.isFinal()])
    jets = jet_selector(jet_def(parts))
    for j in jets:
        all_jets.append(j)

In [ ]:
len(all_jets)